Jupyterlab dependencies, in addition to the normal ones:
```
conda install -c conda-forge jupyterlab nodejs
jupyter labextension install @jupyter-widgets/jupyterlab-manager ipyevents
```

To use `aggdraw` successfully in Windows, clone
https://github.com/ejeschke/aggdraw/ repository, follow
https://stackoverflow.com/questions/17770413/aggdraw-cannot-load-font-no-text-renderer,
and then install the `vectorized-drawing` branch.

To use `aggdraw` successfully in non-Windows:
```
conda install -c conda-forge freetype=2.8.1 aggdraw
```

For Windows users, `conda install -c conda-forge nodejs` will fail with `IOError`. To get around that, install `nodejs` from https://nodejs.org (the LTS version) and `yarn` from https://yarnpkg.com (the stable version).

In [ ]:
from core import ImageWidget

In [ ]:
from ginga.misc.log import get_logger

logger = get_logger('my viewer', log_stderr=True,
                    log_file=None, level=30)

In [ ]:
w = ImageWidget(logger=logger)

In [ ]:
#filename = '../../../stginga/test_data/candels_big_mosaic.fits'
#numhdu = 0
#data_unit = None

filename = '../../../stginga/test_data/jb5g05ubq_flt.fits'
numhdu = 4
data_unit = 'electron'

filename = '/Users/mcraig/Documents/Research/observatory-update-2017-18/tracking_rate/2018-04-16/ey-uma-0001r.fit'
numhdu = 0

# Loads a FITS file
w.load_fits(filename, numhdu=numhdu)

# Loads NDData
#from astropy.nddata import CCDData
#ccd = CCDData.read(filename, hdu=numhdu, unit=data_unit)
#w.load_nddata(ccd)

# Loads array (no WCS)
#from astropy.io import fits
#with fits.open(filename, memmap=False) as pf:
#    arr = pf[numhdu].data.copy()
#w.load_array(arr)

Ginga key bindings documented at http://ginga.readthedocs.io/en/latest/quickref.html . Note that not all documented bindings would work here. Please use an alternate binding, if available, if the chosen one is not working.

Here are the ones that worked during testing with Firefox 52.8.0 on RHEL7 64-bit:

Key | Action | Notes
--- | --- | ---
`+` | Zoom in |
`-` | Zoom out |
Number (0-9) | Zoom in to specified level | 0 = 10
Shift + number | Zoom out to specified level | Numpad does not work
` (backtick) | Reset zoom |
Space > `q` > arrow | Pan |
ESC | Exit mode (pan, etc) |
`c` | Center image
Space > `d` > up/down arrow | Cycle through color distributions
Space > `d` > Shift + `d` | Go back to linear color distribution
Space > `s` > Shift + `s` | Set cut level to min/max
Space > `s` > Shift + `a` | Set cut level to 0/255 (for 8bpp RGB images)
Space > `s` > up/down arrow | Cycle through cuts algorithms
Space > `l` | Toggle no/soft/normal lock |

**TODO: Check out Contrast Mode next**

A viewer will be shown after running the next cell.
In Jupyter Lab, you can split it out into a separate view by right-clicking on the viewer and then select
"Create New View for Output". Then, you can drag the new
"Output View" tab, say, to the right side of the workspace. Both viewers are connected to the same events.

In [ ]:
w

This next cell captures print outputs. You can pop it out like the viewer above. It is very convenient for debugging purpose.

In [ ]:
# Capture print outputs from the widget
display(w.print_out)

The following cell changes the visibility or position of the cursor info bar. For the new setting to take effect, you need to re-run the cell that calls `w` above.

In [ ]:
w.cursor = 'top'  # 'top', 'bottom', None
print(w.cursor)

The rest of the calls demonstrate how the widget API works. Comment/uncomment as needed. Feel free to experiment.

In [ ]:
# Programmatically center to (X, Y) on viewer
#w.center_on((1, 1))

# Programmatically offset w.r.t. current center
#w.offset_to(4096, 2048)

# Programmatically center to SkyCoord on viewer
#w.center_on(SkyCoord('00h14m28.28s', '-30d23m42.66s', frame='icrs'))

# Programmatically offset (in degrees) w.r.t. SkyCoord center
#w.offset_to(0.001, 0.001, skycoord_offset=True)

# Show zoom level
#print(w.zoom_level)

# Programmatically zoom image on viewer
#w.zoom(2)

# Capture what viewer is showing and save RGB image
# Need https://github.com/ejeschke/ginga/pull/665 to work
#w.save('test.png')

In [ ]:
# Get all available image stretch options
print(w.stretch_options)

# Get image stretch algorithm in use
print(w.stretch)

# Change the stretch
w.stretch = 'linear'
print(w.stretch)

In [ ]:
# Get all available image cuts options
print(w.autocut_options)

# Get image cuts algorithm in use
print(w.cuts)

# Change the cuts by providing explicit low/high values
w.cuts = (0, 100)
print(w.cuts)

# Change the cuts with an autocut algorithm
w.cuts = 'zscale'
print(w.cuts)

In [ ]:
# This enables click to center.
#w.click_center = True

# Turn it back off so marking (next cell) can be done.
w.click_center = False

In [ ]:
# This enables/disabled marking mode.
# Set to True, click on viewer to mark.
# When done, set back to False.
w.is_marking = True
#w.is_marking = False
print(w.is_marking)

In [ ]:
# Get table of markers
markers_table = w.get_markers()
print(markers_table)

# For sanity check with written values while marking
print()
for c in markers_table['coord'][:2]:
    print(c.to_string('hmsdms'))

In [ ]:
# Erase markers
w.reset_markers()

In [ ]:
# Programmatically re-mark from table using X, Y.
# To be fancy, first 2 points marked differently.
w.marker = {'type': 'circle', 'color': 'red', 'radius': 50}
w.add_markers(markers_table[:2])
w.marker = {'type': 'circle', 'color': 'cyan', 'radius': 20}
w.add_markers(markers_table[2:])

# TODO: Make this work
# Programmatically re-mark from table using SkyCoord
#w.add_markers(markers_table, use_skycoord=True)

In [ ]:
# Stop marking AND clear markers
w.stop_marking(clear_markers=True)
print(w.is_marking)

In [ ]:
import numpy as np
from astropy.table import Table

# Generate random "stars" to mark
max_stars = 1000
dpix = 20
img = w._viewer.get_image()
bad_locs = np.random.randint(dpix, high=img.shape[1] - dpix, size=[max_stars, 2])

# Only want those not near the edges
mask = ((dpix < bad_locs[:, 0]) & (bad_locs[:, 0] < img.shape[0] - dpix) &
        (dpix < bad_locs[:, 1]) & (bad_locs[:, 1] < img.shape[1] - dpix))

# Put them in table
locs = bad_locs[mask]
t = Table([locs[:, 1], locs[:, 0]], names=('x', 'y'))
print(t)

In [ ]:
# Mark those stars
w.add_markers(t)

In [ ]:
# Define a function to control marker display
def show_circles(n):
    """
    Show and hide circles instead of reconstructing them.
    """
    w.reset_markers()
    t2show = t[:n]
    w.add_markers(t2show)

In [ ]:
# import ipywidgets as ipyw
from IPython.display import display
from ipywidgets import interact

# Initialize and display the slider right below the viewer.
interact(
    show_circles,
    n=ipyw.IntSlider(min=0,max=len(t),step=1,value=0));